In [31]:
import json, tsai, builtins
from torch import Tensor
from numpy import ndarray
from pandas import DataFrame
from cytoolz.dicttoolz import valmap, valfilter, itemfilter, itemmap, merge
from typing import *
from faux.features.ta.loadout import Indicators, bind_indicator
from functools import *
from cytoolz.functoolz import *
import cytoolz.functoolz as ft
import pandas as pd
import pandas_ta as ta


def transform_json(json_data, variables={}, functions={}, path=[]):
    if isinstance(json_data, dict):
        return {k: transform_json(v, variables, functions, path + [k]) for k, v in json_data.items()}
    elif isinstance(json_data, list):
        return [transform_json(v, variables, functions, path + [i]) for i, v in enumerate(json_data)]
    elif isinstance(json_data, str):
        if "${" in json_data and "}" in json_data:
            return transform_json(json_data.format(**valmap(str, variables)))
        elif json_data == "?":
            return transform_json(input("Enter value for {}:".format(".".join(map(str, path)))), variables, functions, path)
        elif json_data.startswith("`") and json_data.endswith("`"):
            expr = json_data[1:len(json_data)-1]
            #TODO evaluate by parsing the expression to allow for some magic
            # print(json_data[1:][:-1]
            # return eval(expr, merge(functions, variables))
            return json_data
        else:
            return json_data
    else:
        return json_data
     
def parse_feature_spec(spec: List[Any])->Callable[[DataFrame], DataFrame]:
   analysis_funcs = list(map(parse_feature_spec_item, spec))
   
   def wfunc(df: DataFrame)->DataFrame:
      return reduce(lambda df, fn: fn(df), analysis_funcs, df)
   
   return wfun

def lookup_fn(fn_lookup, fn_id):
   try:
      fn = bind_indicator(fn_id)
      return fn
   except Exception as error:
      pass
   raise KeyError(fn_id)

def bind_item_fn(func, configuration={}):
   @wraps(func)
   def wfunc(df:DataFrame, *args, **kwargs):
      return func(df, *args, **kwargs)
   return wfunc

class TAFnLookup:
   def __init__(self) -> None:
      self.cache = dict()
      
   def mount_pinescript(self, pinescript_or_filename:str):
      """ expose the definitions in the given pinescript as functions """
      #TODO
      return None
      
   def __getitem__(self, fn_id:str):
      # raise Exception(fn_id)
      if not isinstance(fn_id, str):
         return None
      
      elif fn_id in self.cache:
         return self.cache[fn_id]
      elif isinstance(fn_id, str) and hasattr(ta, fn_id):
         ta_fn = getattr(ta, fn_id)
         if callable(ta_fn):
            self.cache[fn_id] = ta_fn
            return ta_fn
      elif isinstance(fn_id, str) and hasattr(pd.Series, fn_id):
         series_method = getattr(pd.Series, fn_id)
         if callable(series_method):
            self.cache[fn_id] = series_method
            return series_method
      return None

fn_lookup = TAFnLookup()
import re

_uid = [0]
def anoncolid():
   # nonlocal _uid
   name = f'Unnamed {_uid[0]}'
   _uid[0] += 1
   return name

def parse_feature_spec_item(item:Any)->Callable[[DataFrame], DataFrame]:
   type_name = type(item).__qualname__
   if isinstance(item, str):
      if (item[0], item[-1]) == ('`', '`'):
         expr = item[1:-1]
         
         #TODO when the expr-string represents a column-assignment, return an eval_function that invokes eval inline
         assign_pat = re.compile(r'^\s*([a-zA-Z_]\w*)\s*=\s*')
         m = assign_pat.match(expr)
         if m is not None:
            #* Named assignments
            def eval_fn(df: DataFrame):
               print(f'Evaluating {expr} on DataFrame..')
               df = df.copy()
               df.eval(expr, inplace=True)#, resolvers=(fn_lookup,))
               return df
            return eval_fn
         
         def eval_fn(df: DataFrame)->DataFrame:
            print(f'Evaluating {expr} on DataFrame..')
            result = df.eval(expr)#, resolvers=(fn_lookup,))
            df[anoncolid()] = result
            return df
         
         return eval_fn
      
   elif isinstance(item, list):
      if len(item) == 1:
         raise NotImplemented()
      
   elif isinstance(item, dict):
      if len(item) == 1:
         item_ident, item_args = next(iter(item.items()))
         item_fn = lookup_fn(fn_lookup, item_ident)
         item_wfn = bind_item_fn(item_fn, item_args)
         return item_wfn
      
   elif callable(item):
      return item
   
   raise ValueError(f'Unhandled {item}')

the_functions = merge(builtins.__dict__, {})
# print('cunt-lips:'[-1])
# input()

from faux.backtesting.common import load_frame
from tools import dotget, dotgets



In [38]:
json_data = {
   'name': 'Example (Proto)Strategy #1',
   'description': 'This is a test',
   
   'data': {
      'features': [
         # '`vol_delta=volume.pct_change()`',
         # '`z_close_delta = close.pct_change().zscore()`',
         # '`trend_a = close.pct_change().sma(5).zscore()`',
         # {'zscore': {'close': 'vol_delta'}}
         '`close.pct_change()`',
         '`close.pct_change().zscore()`',
         '`pissAndTitties = (high - low)`',
         '`titsAndPiss=pissAndTitties.zscore(30)`'
      ]
   }
}

for name in dir(ta):
   val = getattr(ta, name)
   if callable(val) and not hasattr(pd.Series, name):
      setattr(pd.Series, name, val)

df = load_frame('AAPL', './stonks')
features = transform_json(json_data)['data']['features']
print(features)

class Analysis:
   def __init__(self, *items):
      self.indicators = list(map(parse_feature_spec_item, items))
      self._fn = (lambda df:  reduce(lambda df, fn: fn(df), self.indicators, df))
      
   def __call__(self, df:DataFrame, **kwargs)->DataFrame:
      input_df:DataFrame = df.copy()
      output_df:DataFrame = self._fn(input_df)
      # print(input_df)
      print(output_df)
      return output_df

feat = Analysis(*features)
print(feat)

print(feat(df)[[]])


1562758150.py:26 ['`close.pct_change()`', '`close.pct_change().zscore()`', '`pissAndTitties = (high - low)`', '`titsAndPiss=pissAndTitties.zscore(30)`']
1562758150.py:41 <__main__.Analysis object at 0x7f141431db80>
3002217508.py:115 Evaluating close.pct_change() on DataFrame..
3002217508.py:115 Evaluating close.pct_change().zscore() on DataFrame..
3002217508.py:108 Evaluating pissAndTitties = (high - low) on DataFrame..
3002217508.py:108 Evaluating titsAndPiss=pissAndTitties.zscore(30) on DataFrame..
1562758150.py:37 
                                datetime    open    high     low   close  \
datetime                                                                  
2013-01-28 15:00:00 2013-01-28 15:00:00   15.64   16.19   15.57   16.07   
2013-01-29 15:00:00 2013-01-29 15:00:00   16.38   16.44   16.15   16.37   
2013-01-30 15:00:00 2013-01-30 15:00:00   16.32   16.52   16.23   16.32   
2013-01-31 15:00:00 2013-01-31 15:00:00   16.32   16.40   16.25   16.27   
2013-02-01 15:00:00 2013-